If you are new to dashDB and it's integrated Apache Spark capabilities you may first want to check out this [overview article](http://www.ibmbigdatahub.com/blog/evolving-enterprise-data-warehouse-beyond-sql-apache-spark).

## Demo setup

In [1]:
/*
 * © Copyright IBM Corporation 2016 All rights reserved.
 * LICENSE: BSD-3, https://opensource.org/licenses/BSD-3-Clause
 */

import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.sql.SQLContext
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.types.DoubleType
import org.apache.spark.sql.types.StructType
import org.apache.spark.sql.types.StructField
import org.apache.spark.sql.Row
import org.apache.spark.sql.types.IntegerType
import java.sql.Connection
import java.sql.SQLException
import java.sql.ResultSet
import java.sql.Statement
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.clustering.KMeansModel
import org.apache.spark.ml.feature.VectorAssembler

Load the brunel visualitzation lib for Scala. <b>Note</b> that the brunelvis.org server is often not accessible from certain locations, e.g. from softlayer machines. In that case download the library by hand and upload to dashDB via __`spark-submit.sh --upload-files`__.

In [51]:
//NOT-FOR-APP
//Load the library directly from the repository:
%AddJar -magic https://brunelvis.org/jar/spark-kernel-brunel-all-2.0.jar

//Use the following lines (adapt user name) if access to repository is not working due to some local firewall constraints.
//%AddJar -magic file:/mnt/blumeta0/home/bluadmin/spark/apps/spark-kernel-brunel-all-2.0.jar -f

Starting download from file:/mnt/blumeta0/home/bluadmin/spark/apps/spark-kernel-brunel-all-2.0.jar
Finished download of spark-kernel-brunel-all-2.0.jar


The following cell prepares the input data based on sample tables that are available in every dashDB system out of the box. So you can run this notebook against any dashDB instance that you have available. But when the cell runs for the first time the setup can take a couple of minutes.

In [52]:
//NOT-FOR-APP
import java.sql.DriverManager
import java.sql.Connection
val connection = DriverManager.getConnection("jdbc:db2:BLUDB")
try {   
    val statement = connection.createStatement()
    statement.executeUpdate("create table tornado as ( " +
         " select objectid,yr,mo,dy, time, cast(slat as float) as slat,cast(slon as float) as slon, " +
         "        elat,elon,cast (len as float) as len,cast(wid as float) as wid,st,cast(mag as float) as mag " +
         " from samples.geo_tornado ) with data")
} catch {
    case e: SQLException => { if (e.getErrorCode() == -601) println("Table TORNADO already exists") }
}
try {   
    val statement = connection.createStatement()
    statement.executeUpdate("create table texas_customers as ( " +
         " select b.objectid, b.name, b.insurance_value, db2gse.st_x(b.shape) lon, db2gse.st_y(b.shape) lat " +
         "     from samples.geo_county a, samples.geo_customer b " +
         "     where a.statefp='48' and db2gse.st_within(b.shape, a.shape) = 1 " +
         " ) with data")
} catch {
    case e: SQLException => { if (e.getErrorCode() == -601) println("Table TEXAS_CUSTOMERS already exists") }
}
connection.close()

Table TORNADO already exists
Table TEXAS_CUSTOMERS already exists


## Data acquisition

Read the training data using the IDAX data source. In case of dashDB MPP systems this will give us
automatically a DataFrame with exactly the number of partitions as there are MLNs (database partitions)
in dashDB and the DataFrame partitions will be located in each according executor colocated with the
dashDB MLN.

In [3]:
var input = sqlContext.read.
   format("com.ibm.idax.spark.idaxsource").
   option("url", "jdbc:db2:BLUDB").
   option("dbtable", "TORNADO").
   load()

The input data are all recorded tornados in USA since 1950. It is taken from http://www.spc.noaa.gov/.

When this demo is run with a dashDB SMP system (i.e. non partitioned databse) the next step should be run in order to achieve parallel computation. You should skip this step in case you run against a dashDB MPP because then the data frame is partitioned automatically for you.

In [4]:
// (4) Repartition to 10 partitions
// not needed for MPP
input = input.repartition(10)

## Data exploration

Let's take a look at the Tornado data now:

In [5]:
//NOT-FOR-APP
input.show()

val density = input.select("ST").groupBy("ST").count()
density.orderBy(org.apache.spark.sql.functions.col("count").desc).show()


+--------+----+---+---+--------+-----+-------+-----+-------+----+-----+---+----+
|OBJECTID|  YR| MO| DY|    TIME| SLAT|   SLON| ELAT|   ELON| LEN|  WID| ST| MAG|
+--------+----+---+---+--------+-----+-------+-----+-------+----+-----+---+----+
|     124|1950|  5| 31|15:30:00|35.42| -77.25|  0.0|    0.0| 0.1| 10.0| NC| 1.0|
|     134|1950|  6|  9|18:25:00| 36.8| -97.25|  0.0|    0.0| 0.1|100.0| OK| 1.0|
|     144|1950|  6| 19|20:40:00|36.08|-102.53|36.08|-102.25|15.8|167.0| TX| 0.0|
|     154|1950|  7|  1|12:00:00| 39.3| -96.92|  0.0|    0.0| 0.1| 10.0| KS| 1.0|
|     164|1950|  7| 15|17:30:00| 41.8|  -96.6|41.73| -96.42|10.0|440.0| NE| 4.0|
|     174|1950|  7| 26|15:30:00|35.53|  -77.4|  0.0|    0.0| 0.1| 10.0| NC| 1.0|
|     184|1950|  8| 24|03:45:00| 34.4| -91.43|34.43| -91.38| 3.6|200.0| AR| 2.0|
|     194|1950| 11|  4|17:00:00| 40.2| -76.12| 40.4| -75.93|15.9|100.0| PA| 3.0|
|     204|1951|  2|  1|13:00:00|37.37| -78.78|  0.0|    0.0| 0.1| 50.0| VA| 1.0|
|     214|1951|  3|  2|16:00

For visualization we sample the data down a bit:

In [6]:
//NOT-FOR-APP
val inputSample = input.sample(false,0.1)

Now let's visualize all sampled tornado start coordinates on a map:

In [7]:
%%brunel map('usa') + data('inputSample') x(slon) y(slat) title("Input Data Density per State") :: width=800, height=400
        

Let's visualize tht as a density map summed up per state:

In [8]:
%%brunel map('usa') + data('density') map color(count:[blues, reds]) key(st) label(st) title("Input Data Density per State")  :: width=800, height=800

Let's focus on Texas as the state with highest density of tornados. So let's filter the tornado records for Texas only and see how many we have:

In [9]:
val input_texas = input.filter("ST = 'TX'")
input_texas.count()

8197

## Machine Learning
Let's use machine learning to cluster the hot spots for tornados within Texas. We use a classical clustering ML algorithm.

First we define the feature selection stage of the Spark ML pipeline. We train based on Tornado latitude and logitude.

In [10]:
val assembler = new VectorAssembler().
            setInputCols(Array("SLAT", "SLON")).
            setOutputCol("features")

Now we define the model training stage for unsupervised cluster training using KMeans algorithm from Spark ML API. You can change the number of iterations as well as the number of cluster centers that should be detected:

In [11]:
val clustering = new KMeans().
            setFeaturesCol("features").
            setK(5).
            setMaxIter(5).
            setSeed(123)

Now we stitch together the ML pipeline with feature selection and training stages:

In [12]:
val pipe = new Pipeline().setStages(Array(assembler, clustering))

Now run the ML pipeline on the input data of Texas tornados:

In [13]:
val model = pipe.fit(input_texas)

This ran quick. If you want to observe a longer model training run, execute the following cell:

In [10]:
var input_ballooned = input.unionAll(input).unionAll(input).unionAll(input).unionAll(input).unionAll(input).unionAll(input)
pipe.fit(input_ballooned)

pipeline_4202f74f51ab

Let's explore the trained model and print out the detected cluster centers with their latitude and lognitude:

In [14]:
//NOT-FOR-APP
val clustercenters = model.stages(1).asInstanceOf[KMeansModel].clusterCenters
val clustercentersschema = StructType(for (i <- 1 to clustercenters.head.size) yield StructField(s"col$i", DoubleType, false))
val clustercentersconverted = clustercenters.map { x: Vector => Row.fromSeq(x.toArray) }
val clustercentersdataframe = sqlContext.createDataFrame(sc.parallelize(clustercentersconverted), clustercentersschema).toDF("SLAT", "SLON")
clustercentersdataframe.show()

+------------------+-------------------+
|              SLAT|               SLON|
+------------------+-------------------+
| 32.70784907638543| -96.89023065401896|
|32.082328617992175|-101.02475026075619|
|28.785211514683144| -97.81785340030912|
| 34.75709166666667|-101.46438628691983|
|30.430661156462584| -95.05999394557824|
+------------------+-------------------+



Let's plot that on a map:

In [15]:
%%brunel map('usa') + data('clustercentersdataframe') x(slon) y(slat)  title("Tornado Cluster Centers")

Comparing to official NOAA Tornado density maps confirms that our cluster centers make sense.

![title](http://www.spc.noaa.gov/wcm/2015-wbc-anoms.png)

Now we test the model by simply assigning (scoring) the clusters for each recorded tornado in Texas and then aggregating and ranking each cluster by the number of assigned tornados:

In [16]:
//NOT-FOR-APP
val output_train = model.transform(input_texas).select("OBJECTID","SLON","SLAT","prediction")
val tornados_with_clusters = output_train.toDF(Seq("OBJECTID","SLON","SLAT","ClusterID"): _*)

tornados_with_clusters.groupBy("ClusterID").count().orderBy(org.apache.spark.sql.functions.col("count").desc).show()

+---------+-----+
|ClusterID|count|
+---------+-----+
|        0| 1984|
|        3| 1892|
|        1| 1527|
|        4| 1497|
|        2| 1297|
+---------+-----+



## Making some real predictions

We want to assign each holder of an insurance policies for real estate in Texas to one of the tornado clusters in order to caclulate a risk score according to which cluster his property belongs to.

First we read in insurance holder records for Texas (this is mock up data for the sake of this demo notebook):

In [17]:
var insurance_holders = sqlContext.read.
   format("com.ibm.idax.spark.idaxsource").
   option("url", "jdbc:db2:BLUDB").
   option("dbtable", "TEXAS_CUSTOMERS").
   load()
insurance_holders = insurance_holders.repartition(10)

Now let's take a look at the insurance holder data:

In [18]:
//NOT-FOR-APP
var insurance_holders_sample = insurance_holders.sample(false,0.05)
insurance_holders.show()
insurance_holders.count()

+--------+----------------+---------------+-------------------+------------------+
|OBJECTID|            NAME|INSURANCE_VALUE|                LON|               LAT|
+--------+----------------+---------------+-------------------+------------------+
|     577|     Eric Dantin|         127837|-100.58847661550334| 29.74588991029117|
|    1393|  Nicolas Elwood|         185688| -94.53209248517999|31.403126427742507|
|    1243|       Kay Islas|         227075|  -94.7047010220345|31.291715422428865|
|    1253|         Pia Gan|         273798| -94.75725025099825|31.522454518461302|
|     556|     Blake Groll|         216180|-100.52232121774995|29.904393270632966|
|    1323|Johnathon Tunney|         156433| -94.86915810925927|  31.6740243522421|
|     566|Kemberly Pereyra|         170307|-100.68748862999627|30.094364245713393|
|    1163| Evette Saladino|        1497101| -94.37718879802458|31.731364291329044|
|    1331|    Elva Garrido|         116033| -94.76758586429617| 31.58255154961438|
|   

83663

Let's visualize a sample of the Texas insurance holders on a map:

In [19]:
%%brunel map('usa') + data('insurance_holders_sample') x(lon) y(lat) title("Customers in Texas") :: width=800, height=400 

Finally we run a prediction with the tornado cluster model on the insurance holders to assign each one to its according tornado cluster. Then we print a sample of the result.

In [20]:
val insurance_holders_renamed = insurance_holders.toDF(Seq("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON", "SLAT"): _*)
var insurance_holders_scored = model.transform(insurance_holders_renamed).select("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON","SLAT","prediction")
insurance_holders_scored = insurance_holders_scored.toDF(Seq("OBJECTID", "NAME", "INSURANCE_VALUE", "SLON", "SLAT", "ClusterID"): _*)

insurance_holders_scored.show(10)
//insurance_holders_scored.groupBy("ClusterID").count().show()

println("Writing results to table TEXAS_CUSTOMERS_SCORED ...")
insurance_holders_scored.write.format("com.ibm.idax.spark.idaxsource").
   option("url", "jdbc:db2:BLUDB").
   option("dbtable", "TEXAS_CUSTOMERS_SCORED").
   mode(SaveMode.Overwrite).
   save()
println("Done")

+--------+----------------+---------------+-------------------+------------------+---------+
|OBJECTID|            NAME|INSURANCE_VALUE|               SLON|              SLAT|ClusterID|
+--------+----------------+---------------+-------------------+------------------+---------+
|     577|     Eric Dantin|         127837|-100.58847661550334| 29.74588991029117|        1|
|    1393|  Nicolas Elwood|         185688| -94.53209248517999|31.403126427742507|        4|
|    1243|       Kay Islas|         227075|  -94.7047010220345|31.291715422428865|        4|
|    1253|         Pia Gan|         273798| -94.75725025099825|31.522454518461302|        4|
|     556|     Blake Groll|         216180|-100.52232121774995|29.904393270632966|        1|
|    1323|Johnathon Tunney|         156433| -94.86915810925927|  31.6740243522421|        4|
|     566|Kemberly Pereyra|         170307|-100.68748862999627|30.094364245713393|        1|
|    1163| Evette Saladino|        1497101| -94.37718879802458|31.7313

The resulting cluster ID determines the risk class that the insurance holder will be assigned to. In addition we could run a fine tuning for the risk assignment by folding in the distance from the insured property to the cluster center.

Let's colorize the sampled Texas insurance holders according to their assigned cluster:

In [21]:
//NOT-FOR-APP
var insurance_holders_scored_sample = insurance_holders_scored.sample(false, 0.05)

In [22]:
%%brunel map('usa') + data('insurance_holders_scored_sample') x(SLON) y(SLAT) title("Texas Customers - Cluster-Relation") color(ClusterID) :: width=800, height=400 

## Deploying as application
You can easily deploy the essential logic of this notebook as an application that can be run at any time via a dashDB [stored procedure](https://www.ibm.com/support/knowledgecenter/SS6NHC/com.ibm.swg.im.dashdb.analytics.doc/doc/r_spark_applications_functions.html) invokation, via dashDB's command line [spark-submit.sh](https://www.ibm.com/support/knowledgecenter/SS6NHC/com.ibm.swg.im.dashdb.doc/learn_how/spark_ref.html) or via a [REST API](https://developer.ibm.com/clouddataservices/wp-content/themes/projectnext-clouddata/dashDBanalytics/#/).

Note that a few cells above have the __`//NOT-FOR-APP`__ comment. Cells with this comment as well as cells that only contain Jupyter cell magic starting with __`%%`__ are automatically skipped when deploying the notebook as an application. This way you can organize your notebook for interactive vs. backend logic.

In order to generate and deploy an application for this notebook select __`File->Deploy as->Deploy to dashDB Spark`__ in the menu above. After a short while it shows you the result page for the deployment with the specific invocation options that you just can copy and past to try them out immediately.